In [2]:
model_dir = "/home/arvid/models/chemformers/2021-10-07/1e-1/"
from Transformer import *
from Transformer import Seq2SeqTransformer
from Tools import *
from modules import *
from constants import *
from Reaction_dataset import *

args = pickle.load(open(model_dir + "params.pickle", "rb"))

DEVICE = "cuda:2"
torch.torch.cuda.set_device(DEVICE)
print("Using device " + str(torch.torch.cuda.current_device()) + "/" + str(torch.cuda.device_count())
      +", name: " + str(torch.cuda.get_device_name(0)))

chemFormer = Seq2SeqTransformer(num_encoder_layers=args["N_ENCODERS"],
                                num_decoder_layers=args["N_DECODERS"],
                                emb_size=args["EMBEDDING_SIZE"],
                                nhead=args["N_HEADS"],
                                src_vocab_size=SRC_VOCAB_SIZE,
                                tgt_vocab_size=TGT_VOCAB_SIZE,
                                dim_feedforward=args["DIM_FF"],
                                dropout=args["DROPOUT"],
                                DEVICE=DEVICE).to(DEVICE)

chemFormer.load_state_dict(torch.load(model_dir + "/weights_last_epoch"))

file = open("/home/arvid/data/USTPO_paper_5x/USTPO_5x_parsed.pickle",'rb')
data = pickle.load(file)
if False: 
        data = {"train": data["train"][0:64], "eval": data["eval"][0:64]}
datasets = {}
dataloaders = {}
for split in ['eval', "train"]:
    datasets[split] = ReactionDataset(data=data,
                                        split=split,
                                        args=args)

    dataloaders[split] = DataLoader(datasets[split],
                                    batch_size=args["BATCH_SIZE"],
                                    shuffle=(split != 'test'),
                                    num_workers=8,
                                    pin_memory=False,
                                    drop_last=True)
    
chemFormer.eval()




Using device 2/16, name: Tesla V100-SXM3-32GB


Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_feature

In [3]:
# Code always computes embedding, could be more efficient.
split = "eval"

# Init set and put all ixs from eval to be inferred
Q = set()
#for i in range(len(datasets[split])): Q.add(i)
for i in range(1000): Q.add(i)
inferred = {}
print(len(datasets[split]))

# Init src and tgt tensors, free_ixs_tracker & element tracker
capacity = 32
src = torch.zeros([MAX_SEQ_LEN, capacity]).to(DEVICE)
tgt = torch.zeros([MAX_SEQ_LEN, capacity]).to(DEVICE)
element_tracker = {}
free_ixs_tracker = set()
for e in range(capacity): free_ixs_tracker.add(e)
j = 0
# Infer while PQ nonempty
while Q != set() or len(free_ixs_tracker) != capacity:
    j += 1 
    print(j)
    # Track elements and fill a batch
    if Q != set():
        while free_ixs_tracker != set():
            # Pop data item for a free ix in tensor, and record where it is put & ix where to get next word
            free_ix = free_ixs_tracker.pop()
            data_ix = Q.pop()
            element_tracker[free_ix] = [data_ix, 0]
            data = torch.tensor(datasets[split].__getitem__(data_ix)["ps"]).to(DEVICE)
            src[:,free_ix] = data
            tgt[:,free_ix] = PAD_IDX
            tgt[0,free_ix] = BOS_IDX
        
    # Calculate all masks
    src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask = create_mask(src=src,
                                                                         tgt=tgt,
                                                                         DEVICE=DEVICE)
    # Compute embedding
    memory = chemFormer.encode(src=src,
                    src_mask=src_mask,
                    src_key_padding_mask=src_key_padding_mask)
    
    # Decode until a sequence finishes
    while True:
        
        # Calculate all masks again
        src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask = create_mask(src=src,
                                                                                     tgt=tgt,
                                                                                     DEVICE=DEVICE)
        
        logits = chemFormer.decode(tgt=tgt,
                                   memory=memory,
                                   tgt_mask=tgt_mask,
                                   memory_key_padding_mask=src_key_padding_mask,
                                   tgt_key_padding_mask=tgt_key_padding_mask)
        probs = chemFormer.generator(logits)

        # Iterate over tracked seqs
        should_break = False
        for k,v in element_tracker.items():
            if element_tracker[k][0] in inferred: continue
            # Find next word for each seq, record that seq expands
            if element_tracker[k][1] == MAX_SEQ_LEN - 1:
                inferred[element_tracker[k][0]] = None
                free_ixs_tracker.add(k)
                should_break = True
            
            else:
                next_word = torch.argmax(probs[element_tracker[k][1], k,:])

                element_tracker[k][1] += 1

                tgt[element_tracker[k][1], k] = next_word

                # If seq ends, need to save result & remove from capacity
                if next_word == EOS_IDX:
                    seq = tgt[:,k]
                    seq = tokens_to_smiles(seq[torch.where(seq != PAD_IDX)][1:-1].cpu().numpy())
                    gt = datasets[split].__getitem__(element_tracker[k][0])["rs"]
                    gt = tokens_to_smiles(gt[np.where(gt != PAD_IDX)][1:-1])
                    
                    inferred[element_tracker[k][0]] = {"pred": seq, "tgt": gt}
                    free_ixs_tracker.add(k)
                    should_break = True
                
        if should_break == True: break
    

9990
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276

In [4]:
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') 

TP = 0
FP = 0
for k,v in inferred.items():
    if v is None:
        FP += 1
        continue
    out_smiles = v["pred"]
    tgt_smiles = v["tgt"]
    out_mols = [Chem.MolFromSmiles(e) for e in out_smiles]
    tgt_mols = [Chem.MolFromSmiles(e) for e in tgt_smiles]
    if None in out_mols:
        FP += 1
    else:
        out_smiles_canonical = [Chem.MolToSmiles(e) for e in out_mols]
        tgt_smiles_canonical = [Chem.MolToSmiles(e) for e in tgt_mols]

        if set(out_smiles_canonical) == set(tgt_smiles_canonical): TP += 1
        else: FP += 1

print(TP/(TP+FP))

0.184


In [ ]:
inferred[np.random.randint(100)]

In [ ]:
from Tools import tokens_to_smiles
# greedy_decode will be used in a later stage to test our model
def greedy_decode(model, src, src_mask, max_len, start_symbol, DEVICE):
    src = src.to(DEVICE).unsqueeze(-1)
    #print(src.shape)
    src_mask = src_mask.to(DEVICE)
    #print(src_mask.shape)
    memory = model.encode(src, src_mask)
    #print(memory.shape)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0), DEVICE)
                    .type(torch.bool))
        #print(tgt_mask.shape)
        
        out = model.decode(ys, memory, tgt_mask)
        #print(out.shape)
        probs = model.generator(out)
        #print(probs.shape)
        next_word = torch.argmax(probs, axis=2)[-1,:].squeeze()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        #print(ys.shape)
        if next_word == EOS_IDX:
            break
    return ys.squeeze(-1).cpu().numpy()

i=np.random.randint(1000)

data = datasets["eval"].__getitem__(i)
src = torch.tensor(data["ps"]).to(dtype=torch.int64)
tgt = data["rs"]
tgt = tgt[np.where(tgt != PAD_IDX)][1:-1]
src = src[torch.where(src != PAD_IDX)]
num_tokens = src.shape[0]
src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
out_tokens = greedy_decode(chemFormer,
                           src,
                           src_mask,
                           max_len=MAX_SEQ_LEN,
                           start_symbol=BOS_IDX,
                           DEVICE=DEVICE).flatten()[1:-1]
out_smiles = tokens_to_smiles(out_tokens)
tgt_smiles = tokens_to_smiles(tgt)
print(out_smiles)
print(tgt_smiles)
out_mols = [Chem.MolFromSmiles(e) for e in out_smiles]
tgt_mols = [Chem.MolFromSmiles(e) for e in tgt_smiles]
if None in out_mols:
    print("FALSE")
else:
    out_smiles_canonical = [Chem.MolToSmiles(e) for e in out_mols]
    tgt_smiles_canonical = [Chem.MolToSmiles(e) for e in tgt_mols]
    print(out_smiles_canonical)
    print(tgt_smiles_canonical)

    if set(out_smiles_canonical) == set(tgt_smiles_canonical): print("TRUE")
    else: print("FALSE")
print()
